In [1]:
import requests 

import numpy as np
import pandas as pd
from tqdm import tqdm

from elasticsearch import Elasticsearch

In [2]:
from sentence_transformers import SentenceTransformer

In [3]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

In [4]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

# Download dataset

In [5]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [6]:
ml_documents = [doc for doc in documents if doc["course"] == "machine-learning-zoomcamp"]

In [7]:
len(ml_documents)

375

# Q1. Getting the embeddings model


In [8]:
model = SentenceTransformer("multi-qa-distilbert-cos-v1")

/usr/local/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
user_question = "I just discovered the course. Can I still join it?"

In [10]:
embeddings = model.encode(user_question)

In [11]:
embeddings[0]

0.07822265

In [12]:
embeddings[0].shape

()

# Q2. Creating the embeddings

In [13]:
ml_documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [14]:
embeddings = []
for doc in tqdm(ml_documents):
    qa_text = f'{doc["question"]} {doc["text"]}'
    embeddings.append(model.encode(qa_text))

100%|██████████| 375/375 [01:00<00:00,  6.15it/s]


In [15]:
X = np.array(embeddings)

In [16]:
X.shape

(375, 768)

# Q3. Search


In [17]:
v = model.encode(user_question)

In [18]:
scores = X.dot(v)

In [19]:
index = np.argmax(scores)
maximum = scores[index]

In [20]:
ml_documents[14]

{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'section': 'General course-related questions',
 'question': 'The course has already started. Can I still join it?',
 'course': 'machine-learning-zoomcamp',
 'id': 'ee58a693'}

In [21]:
index, maximum

(14, 0.65065753)

# Q4. Hit-rate for our search engine

In [22]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

In [23]:
df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [24]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [25]:
len(ground_truth)

1830

In [26]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [27]:
vse = VectorSearchEngine(documents=ml_documents, embeddings=X)

In [28]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q["document"]
    results = vse.search(v_query=model.encode(q["question"]), num_results=5)
    relevance = [d["id"] == doc_id for d in results]
    relevance_total.append(relevance)

100%|██████████| 1830/1830 [05:27<00:00,  5.59it/s]


In [29]:
hit_rate(relevance_total)

0.9398907103825137

# Q5. Indexing with Elasticsearch

In [30]:
def elastic_search_knn(field, vector):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
    }

    search_query = {
        "knn": knn,
        "_source": ["id", "question_text_vector"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [31]:
es_client = Elasticsearch('http://elasticsearch:9200') 

In [32]:
es_client.info()

ObjectApiResponse({'name': '38da6de9601d', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'RjwRqINZR-inwYMXIDoAVw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [33]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "machine-learning-zoomcamp"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'machine-learning-zoomcamp'})

In [34]:
ml_documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [35]:
for doc, embedd in tqdm(zip(ml_documents, X)):
    doc_info = {
        "id": doc["id"], 
        "question_text_vector": embedd
    }
    es_client.index(index=index_name, document=doc_info)

375it [00:01, 295.74it/s]


In [36]:
user_question = "I just discovered the course. Can I still join it?"

In [37]:
user_question_embeddings = model.encode(user_question)

In [38]:
elastic_search_knn("question_text_vector", user_question_embeddings)[0]["id"]

'ee58a693'

In [39]:
[doc for doc in ml_documents if doc["id"] == "ee58a693"]

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'}]

# Q6. Hit-rate for Elasticsearch

In [40]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        question = q['question']
        user_question_embeddings = model.encode(question)
        results = search_function("question_text_vector", user_question_embeddings)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
    }

In [41]:
evaluate(ground_truth, elastic_search_knn)

100%|██████████| 1830/1830 [03:14<00:00,  9.43it/s]


{'hit_rate': 0.9404371584699454}